In [8]:
import numpy as np
import pandas as pd
import requests
import json

#Pick a temporal level to run analysis for:

In [2]:
temporal = 'daily'

In [3]:
start=2020
end=2025

#Location

In [4]:
latitude = 23.777176
longitude = 90.399452

In [5]:
parameters = ['T2M', 'PRECTOT', 'WS10M', 'RH2M']  # Temperature, Precipitation, Wind Speed, Relative Humidity

In [6]:
base_url = "https://power.larc.nasa.gov/api/temporal/{}/point?parameters={}&community=RE&longitude={}&latitude={}&start={}&end={}&format=JSON"
query_url = base_url.format(temporal, ','.join(parameters), longitude, latitude, start, end)
print("Query URL:", query_url)

Query URL: https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,PRECTOT,WS10M,RH2M&community=RE&longitude=90.399452&latitude=23.777176&start=2020&end=2025&format=JSON


#Request Data

In [9]:
%%time
main_response = requests.get(url=query_url, verify=True, timeout=30)
json_response = json.loads(main_response.text)

CPU times: user 60.1 ms, sys: 6.28 ms, total: 66.4 ms
Wall time: 3.68 s


In [10]:
df = pd.DataFrame.from_dict(json_response['properties']['parameter'])

if(temporal == 'hourly'):
    df.index = pd.to_datetime(df.index, format='%Y%m%d%H%M')
elif(temporal == 'daily'):
    df.index = pd.to_datetime(df.index, format='%Y%m%d')

df.replace(-999.0, 0)
df.head()

,WS10M,T2M,RH2M,PRECTOTCORR
2020-01-01,1.49,20.15,76.09,0.00
2020-01-02,1.91,20.13,79.37,0.01
2020-01-03,1.76,17.56,92.62,36.36
2020-01-04,1.48,18.43,83.94,2.04
2020-01-05,2.52,16.85,87.04,6.46


In [19]:
df.replace(-999.0, np.nan, inplace=True)
df.describe().round(3)

,WS10M,T2M,RH2M,PRECTOTCORR
count,2089.000,2089.000,2089.000,2089.000
mean,2.755,25.590,77.165,7.234
std,1.261,4.514,13.846,12.520
min,0.660,13.920,31.570,0.000
25%,1.860,22.180,69.040,0.000
50%,2.420,27.690,81.160,2.330
75%,3.460,28.820,88.650,9.490
max,13.370,33.550,95.950,144.770
